In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter('ignore', RuntimeWarning)

In [2]:
S=pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="C:IH").values
F=pd.read_excel(io=r'../data/8-dataset.xlsx',sheet_name='Loss',usecols="B:IG").values
MT=np.diag(1./(pd.read_excel(io=r'../data/402-dataset.xlsx',sheet_name='Supply',usecols="B").values.flatten()))
supply=np.zeros((402,24,10))
forwa=np.zeros((8,24,10))
for i in range(402):
    supply[i,:,:]=S[i,:].reshape(10,24).T
for j in range(8):
    forwa[j,:,:]=F[j,:].reshape(10,24).T

In [3]:
def GM_11(x):
    n=len(x)
    lamda=x[:-1]/x[1:]
    rang=[lamda.min(), lamda.max()]
    theta=[np.exp(-2/(n+1)),np.exp(2/(n+1))]
    x0=x
    if theta[0]<=rang[0] and rang[1]<=theta[1]:
        c=0
    else:
        while True:
            x0=x0+1
            ratio=x0[:-1]/x0[1:]
            if theta[0]<=ratio.min() and ratio.max()<=theta[1]:
                break
        c=x0[0]-x[0]
    x=x+c
    x1=np.cumsum(x)
    z=0.5*(x1[1:]+x1[:-1])
    B=np.vstack([-z,np.ones(n-1)]).T
    u=np.linalg.pinv(B)@x[1:] 
    sp.var('t')
    sp.var('xx',cls=sp.Function) 
    eq=xx(t).diff(t)+u[0]*xx(t)-u[1]  
    s=sp.dsolve(eq,ics={xx(0):x[0]}) 
    xt=s.args[1]
    if str(xt).replace('.','').isdigit():
        xt=(x[0]-(u[1]/u[0]))*sp.exp(-u[0]*t)+(u[1]/u[0])
    xt=sp.lambdify(t,xt,'numpy') 
    fxt=np.arange(n+1)
    xt1=xt(fxt)
    fo=np.hstack([x[0],np.diff(xt1)]) 
    fo=fo-c
    x=x-c
    epsilon=x-fo[:-1]
    fo=fo[-1]
    return fo,epsilon

In [4]:
def ARIMA_(x):
    x0=x
    n1=len(x)
    di=0
    while True:
            if adfuller(x0)[1]>=0.5:
                x0=np.diff(x0)
                di=di+1
            else:
                break
    n2=len(x0)
    aicbic=np.empty([0,3])
    for i in range(4):
        for j in range(4):
            if  i==0 and j==0:
                continue
            else:
                model=ARIMA(x0, order=(i,0,j))
                try:
                    model=model.fit()
                    aicbic=np.append(aicbic,[[i,j,model.aic+model.bic]],axis=0)
                except BaseException:
                    aicbic=np.append(aicbic,[[i,j,float('inf')]],axis=0)
                    pass
    aicbic[np.isnan(aicbic)]=float('inf')
    index=np.where(aicbic[:,2]==np.min(aicbic[:,2],axis=0))
    mod=ARIMA(x0, order=(int(aicbic[index[0][0],0]),0,int(aicbic[index[0][0],1])))
    fo=mod.fit().forecast(n1+1-n2)
    for i in range(n1-n2):
        fo=np.cumsum(fo)
    x_Fo=fo[-1]
    return x_Fo

In [5]:
S_fo=np.zeros((24,402))
F_fo=np.zeros((24,8))
for k in range(402):
    fc=0
    for l in range(24):
        if np.any(np.delete(supply[k,l,:],[0])==0):
            fc=0
            continue
        fo,eps=GM_11(supply[k,l,:])
        epsilon=ARIMA_(eps)
        fc=fo+epsilon
        if fc<0:
            fc=0
        S_fo[l,k]=fc
data_402 = pd.DataFrame(S_fo)
writer402 = pd.ExcelWriter(r'../data/fo_result_402.xlsx')
data_402.to_excel(writer402, 'page_1', float_format='%.7f')
writer402.save()
writer402.close()
for m in range(8):
    fc=0
    for n in range(24):
        if np.any(np.delete(forwa[m,n,:],[0])==0):
            fc=0
            continue
        fo,eps=GM_11(forwa[m,n,:])
        epsilon=ARIMA_(eps)
        fc=fo+epsilon
        if fc<0:
            fc=0
        F_fo[n,m]=fc
data_8 = pd.DataFrame(F_fo)
writer8 = pd.ExcelWriter(r'../data/fo_result_8.xlsx')
data_8.to_excel(writer8, 'page_1', float_format='%.7f')
writer8.save()
writer8.close()